In [1]:
# Import required packages
import numpy as np
import logging
import pyLDAvis.gensim
import json
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity

from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array

import pandas as pd

In [2]:
data = pd.read_csv('depression.csv')

In [3]:
docs = array(data['text'])

In [4]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [5]:
docs = docs_preprocessor(docs)
#Create Biagram & Trigram Models 
from gensim.models import Phrases
# Add bigrams and trigrams to docs,minimum count 10 means only that appear 10 times or more.
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
#Remove rare & common tokens 
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
dictionary.filter_extremes(no_below=10, no_above=0.2)
#Create dictionary and corpus required for Topic Modeling
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))
print(corpus[:1])

Number of unique tokens: 1343
Number of documents: 16733
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 3)]]


In [6]:
# Set parameters.
num_topics = 5
chunksize = 500 
passes = 20 
iterations = 20
eval_every = 1  

# Make a index to word dictionary.
temp = dictionary[0]  # only to "load" the dictionary.
id2word = dictionary.id2token

lda_model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)
# Print the Keyword in the 5 topics
print(lda_model.print_topics())

[(0, '0.184*"more_than" + 0.095*"more" + 0.094*"mentalhealthmatters" + 0.094*"study" + 0.094*"disorder" + 0.093*"than" + 0.092*"october" + 0.044*"major" + 0.040*"pandemic" + 0.031*"during"'), (1, '0.016*"this" + 0.015*"your" + 0.015*"that" + 0.012*"life" + 0.012*"health" + 0.010*"help" + 0.010*"have" + 0.010*"what" + 0.009*"people" + 0.008*"about"'), (2, '0.073*"research_find" + 0.073*"acupuncture_reduces" + 0.050*"really_help" + 0.049*"acupuncture" + 0.049*"beneficial_good" + 0.049*"have_evidence" + 0.049*"back_acupuncture" + 0.049*"tiba8o94vm_always" + 0.049*"enhances_ssri" + 0.025*"evidence"'), (3, '0.030*"therapy" + 0.028*"online" + 0.027*"online_mindfulness" + 0.019*"mindfulness" + 0.014*"help" + 0.012*"introvert" + 0.011*"introvert_mentalhealthawareness" + 0.011*"shyness_anxietyhelp" + 0.010*"learn" + 0.009*"learn_more"'), (4, '0.100*"disorder_during" + 0.067*"study_published" + 0.066*"million_case" + 0.066*"lancet_october" + 0.066*"document_world" + 0.066*"wide_increase" + 0.066

In [8]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.194790  0.350986       1        1  36.966408
4     -0.334680  0.066054       2        1  33.320727
1      0.209424 -0.079121       3        1  13.644376
0     -0.252952 -0.073993       4        1   9.361590
3      0.183417 -0.263926       5        1   6.706900, topic_info=                                                   Term         Freq  \
term                                                                   
935                                           more_than  4292.000000   
1331                                    disorder_during  8266.000000   
1313                                      research_find  6743.000000   
1303                                acupuncture_reduces  6739.000000   
1297                                    study_published  5522.000000   
1298                                       million_case  5513.000000   
1332                                     document_world  5512.000000   
1337                                      wide_increase  5512.000000   
1334                                     lancet_october  5512.000000   
1339                     2zrvyey9sx_mentalhealthmatters  5510.000000   
331                                               study  2197.000000   
49                                             disorder  2188.000000   
224                                                than  2170.000000   
151                                 mentalhealthmatters  2341.000000   
894                                             october  2149.000000   
14                                                 more  2406.000000   
927                                         really_help  4565.000000   
1302                                        acupuncture  4497.000000   
1309                                      enhances_ssri  4493.000000   
1310                                      have_evidence  4493.000000   
1306                                    beneficial_good  4493.000000   
1304                                   back_acupuncture  4493.000000   
1315                                  tiba8o94vm_always  4493.000000   
667                                               major  2471.000000   
720                                            pandemic  2506.000000   
300                                                case  2765.000000   
1330                                           document  2758.000000   
1292                                             lancet  2757.000000   
1333                       document_world_wide_increase  2757.000000   
1336                     study_published_lancet_october  2757.000000   
...                                                 ...          ...   
737                                 mindfulness_therapy   143.650829   
539              depressiontreatment_depressionrecovery   140.315778   
419                                             contact   138.805129   
781                                       contact_learn   132.898745   
306                       onlinetherapy_onlinetherapist   128.037822   
247                                               visit   127.373885   
1342                           64sp9ppl95_socialanxiety   121.348709   
246                                          overcoming   118.766446   
142                                             session   111.290465   
324                              mentalhealth_introvert   107.315295   
143                                     therapy_session   106.652742   
440                                       socialanxiety   103.048977   
282                                  therapy_overcoming   100.681777   
55                                            effective    99.744777   
1293                                        anxietyhelp    95.641784   
659                                                infj    93.961657   
1317                                      introvert

In [9]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.6561097580361263


In [ ]:
# Compute Coherence Score using UMass
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence="u_mass")
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=docs, start=2, limit=40, step=6)
# Show graph


In [ ]:
import matplotlib.pyplot as plt
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()
plt.show()